In [2]:
import time

import numpy as np

from pycoral.utils.edgetpu import make_interpreter

In [3]:
cls_model_path = "experiments/co_compile_obj_cls/model/efficientnet-M_edgetpu.tflite"
cls_model2_path = "experiments/co_compile_obj_cls/model/tfhub_tf2_resnet_50_imagenet_ptq_edgetpu.tflite"
det_model_path = "experiments/co_compile_obj_cls/model/efficientdet-lite_edgetpu.tflite"

In [4]:
cls_intp = make_interpreter(cls_model_path)
cls_intp.allocate_tensors()
cls2_intp = make_interpreter(cls_model2_path)
cls2_intp.allocate_tensors()
det_intp = make_interpreter(det_model_path)
det_intp.allocate_tensors()


In [12]:
cls_durs = []
for _ in range(100):
    st = time.perf_counter()
    cls_intp.invoke()
    dur = (time.perf_counter() - st) * 1000
    cls_durs.append(dur)

print(np.mean(cls_durs), np.std(cls_durs))

15.955004900042695 0.7445202857184285


In [5]:
cls_durs = []
for _ in range(100):
    st = time.perf_counter()
    cls2_intp.invoke()
    dur = (time.perf_counter() - st) * 1000
    cls_durs.append(dur)

print(np.mean(cls_durs), np.std(cls_durs))

68.22361810987786 2.1449504541036704


In [6]:
det_durs = []
for _ in range(100):
    st = time.perf_counter()
    det_intp.invoke()
    dur = (time.perf_counter() - st) * 1000
    det_durs.append(dur)

print(np.mean(det_durs), np.std(det_durs))

52.4989718200959 4.552529071022282
